## 🧪 Generic Image Augmentation Pipeline

This augmentation pipeline is designed for flexible use in any image classification or clustering context. It supports grayscale or RGB images and applies the following transformations:

- **Flipping**: Random horizontal (50%) and vertical (30%) flips
- **Affine Transformations**: Slight scaling, rotation, and shearing
- **Gaussian Blur**: Soft blurring to simulate lens softness or image noise
- **Additive Noise**: Gaussian pixel-level noise injection
- **Contrast & Brightness Jittering**: Random light/dark balance
- **Elastic Distortion**: Minor spatial warping to mimic drawing inconsistencies

The function iterates over all subdirectories (class labels), applies multiple augmentations per image, and saves the results in a mirrored output structure. Images are resized uniformly (default: 128×128) before augmentation.

> ✅ Useful for testing unsupervised learning, robustness studies, or expanding small visual datasets.

In [1]:
import numpy as np
import imageio
import imgaug.augmenters as iaa
import os
from pathlib import Path
from PIL import Image

AttributeError: `np.sctypes` was removed in the NumPy 2.0 release. Access dtypes explicitly instead.

In [ ]:

# Define augmentation sequence
augmentation_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),                             # horizontal flip
    iaa.Flipud(0.3),                             # vertical flip
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        rotate=(-15, 15),
        shear=(-5, 5)
    ),
    iaa.GaussianBlur(sigma=(0, 1.0)),
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
    iaa.LinearContrast((0.75, 1.25)),
    iaa.Multiply((0.9, 1.1)),                    # brightness
    iaa.ElasticTransformation(alpha=1.0, sigma=0.25)  # slight elastic warp
])

In [ ]:

def augment_images_from_folder(input_dir, output_dir, augmentations_per_image=5, image_size=(128, 128)):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"Processing class directory: {input_dir.name}")
    if input_dir.is_dir():
        out_class_dir = output_dir / input_dir.name
        out_class_dir.mkdir(exist_ok=True)

        for img_path in input_dir.glob("*.png"):
            image = Image.open(img_path).convert("L")  # convert to grayscale
            image = image.resize(image_size)
            image_np = np.array(image)

            images_aug = augmentation_pipeline(images=[image_np for _ in range(augmentations_per_image)])

            for i, aug_img in enumerate(images_aug):
                aug_filename = f"{img_path.stem}_aug{i}.png"
                imageio.imwrite(str(out_class_dir / aug_filename), aug_img)


In [ ]:
augment_images_from_folder(
    input_dir="../data/synthetic",
    output_dir="../data/augmented",
    augmentations_per_image=5,
    image_size=(224, 224)
)


Processing class directory: .gitkeep
Processing class directory: augmented
Processing class directory: synthetic
